In [ ]:
import pandas as pd
import numpy as np

from IPython.display import display
pd.set_option('display.max_columns', None)

import site
site.addsitedir('/Applications/sam-sdk-2015-6-30-r3/languages/python/')
import PySAM.PySSC as pssc

In [ ]:
import ipywidgets as widgets

def createParametersOptionsWidget():
    
    global state_choice, start_year_choice, end_year_choice, system_capacity_choice, state_info
    # Formatting settings for drop-down menus
    style = {'description_width': '200px'}
    layout = widgets.Layout(width='325px')
    
    
    state_info = pd.read_json('USstates_avg_latLong.json')

    # Create drop-down menu 
    state_choice = widgets.Dropdown(options = state_info.state.tolist(), 
                            description = 'State:', style=style, layout=layout)
    start_year_choice = widgets.Dropdown(options = [i for i in range(1998, 2021)],
                                         description = 'Start Year:', style=style, layout=layout)
    end_year_choice = widgets.Dropdown(options = [i for i in range(1998, 2021)],
                                         description = 'End Year:', style=style, layout=layout)
    system_capacity_choice = widgets.Dropdown(options = [i / 10 for i in range(1, 101)],
                                         description = 'Your System Capacity (in MW):', style=style, layout=layout)
    
    
    display(state_choice, start_year_choice, end_year_choice, system_capacity_choice)

In [ ]:
def solar_power_simulation():
    
    global final_data
    appended_data = []
    # data generation
    for year in range(start_year_choice.value, end_year_choice.value+1):
        
        lat = state_info[state_info['state'] == state_choice.value]['latitude'].values[0]
        lon = state_info[state_info['state'] == state_choice.value]['longitude'].values[0]
        api_key = '2aD0f1cpYYogKvIhgzCCEsuBHnVvfGhcaItjJnAU'
        attributes = 'ghi,dhi,dni,wind_speed,air_temperature,solar_zenith_angle'
        year = year
        leap_year = 'false'
        interval = '30'
        utc = 'false'
        your_name = 'Justin+Lin'
        reason_for_use = 'beta+testing'
        your_affiliation = 'HTF'
        your_email = 'slin@wvhtf.org'
        mailing_list = 'false'
    
        url = 'https://developer.nrel.gov/api/nsrdb/v2/solar/psm3-download.csv?wkt=POINT({lon}%20{lat})&names={year}&leap_day={leap}&interval={interval}&utc={utc}&full_name={name}&email={email}&affiliation={affiliation}&mailing_list={mailing_list}&reason={reason}&api_key={api}&attributes={attr}'\
        .format(year=year, lat=lat, lon=lon, leap=leap_year, interval=interval, utc=utc, name=your_name, email=your_email, mailing_list=mailing_list, affiliation=your_affiliation, reason=reason_for_use, api=api_key, attr=attributes)
        
        info = pd.read_csv(url, nrows=1)
        timezone, elevation = info['Local Time Zone'], info['Elevation']
    
        df = pd.read_csv(url, skiprows=2)
        df = df.set_index(pd.date_range('1/1/{yr}'.format(yr=year), freq=interval+'Min', periods=525600/int(interval)))

        # SAM Model for solar simulation
        ssc = pssc.PySSC()
        
        # Resource inputs for SAM model:
        # Must be byte strings
        wfd = ssc.data_create()
        ssc.data_set_number(wfd, b'lat', lat)
        ssc.data_set_number(wfd, b'lon', lon)
        ssc.data_set_number(wfd, b'tz', timezone)
        ssc.data_set_number(wfd, b'elev', elevation)
        ssc.data_set_array(wfd, b'year', df.index.year)
        ssc.data_set_array(wfd, b'month', df.index.month)
        ssc.data_set_array(wfd, b'day', df.index.day)
        ssc.data_set_array(wfd, b'hour', df.index.hour)
        ssc.data_set_array(wfd, b'minute', df.index.minute)
        ssc.data_set_array(wfd, b'dn', df['DNI'])
        ssc.data_set_array(wfd, b'df', df['DHI'])
        ssc.data_set_array(wfd, b'wspd', df['Wind Speed'])
        ssc.data_set_array(wfd, b'tdry', df['Temperature'])
    
        # Create SAM compliant object  
        dat = ssc.data_create()
        ssc.data_set_table(dat, b'solar_resource_data', wfd)
        ssc.data_free(wfd)
    
        # Specify the system Configuration
        # Set system capacity in MW
        ssc.data_set_number(dat, b'system_capacity', system_capacity_choice.value)
        # Set DC/AC ratio (or power ratio). See https://sam.nrel.gov/sites/default/files/content/virtual_conf_july_2013/07-sam-virtual-conference-2013-woodcock.pdf
        ssc.data_set_number(dat, b'dc_ac_ratio', 1.1)
        # Set tilt of system in degrees
        ssc.data_set_number(dat, b'tilt', 25)
        # Set azimuth angle (in degrees) from north (0 degrees)
        ssc.data_set_number(dat, b'azimuth', 180)
        # Set the inverter efficency
        ssc.data_set_number(dat, b'inv_eff', 96)
        # Set the system losses, in percent
        ssc.data_set_number(dat, b'losses', 14.0757)
        # Specify fixed tilt system (0=Fixed, 1=Fixed Roof, 2=1 Axis Tracker, 3=Backtracted, 4=2 Axis Tracker)
        ssc.data_set_number(dat, b'array_type', 0)
        # Set ground coverage ratio
        ssc.data_set_number(dat, b'gcr', 0.4)
        # Set constant loss adjustment
        ssc.data_set_number(dat, b'adjust:constant', 0)
    
        # execute and put generation results back into dataframe
        mod = ssc.module_create(b'pvwattsv5')
        ssc.module_exec(mod, dat)
        df[b'generation'] = np.array(ssc.data_get_array(dat, b'gen'))
        
        # free the memory
        ssc.data_free(dat)
        ssc.module_free(mod)
        
        appended_data.append(df)
    
    final_data = pd.concat(appended_data)
    
    print(f'\033[1mThis dataset has {final_data.shape[0]} rows and {final_data.shape[1]} columns\033[0m')
    return final_data.head(20)

    

In [ ]:
from matplotlib import pyplot as plt
def daily_plot(df):
    # Using a style
    plt.style.use('Solarize_Light2')
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax2 = ax.twinx()
    df = df.groupby('Hour').sum()[['GHI', 'DNI', 'DHI', 'Solar Zenith Angle', b'generation']]
    df[['GHI', 'DNI', 'DHI', 'Solar Zenith Angle']].plot(ax=ax, figsize=(15,8), style={'Solar Zenith Angle': '-o', 'DNI': 'c-o', 'DHI': '-o', 'GHI': 'r-o'}, legend=False)
    df[b'generation'].plot(ax=ax2, style={b'generation': '-o'})
    ax.set_ylabel('W/m2')
    ax2.set_ylabel('kW')
    ax.grid()
    ax.legend(loc=2, ncol=5, frameon=False)
    ax2.legend(loc=1, frameon=False)
    ax.set_xticks([i for i in range(1, 25, 2)])
    if start_year_choice.value != end_year_choice.value:
        plt.title(f'Aggregated Daily solar simulation results in {state_choice.value} from {start_year_choice.value} to {end_year_choice.value}')
    else:
        plt.title(f'Aggregated Daily solar simulation results in {state_choice.value} in {start_year_choice.value}')


In [ ]:
def monthly_plot(df):
    # Using a style
    plt.style.use('Solarize_Light2')
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax2 = ax.twinx()
    df = df.groupby('Month').sum()[['GHI', b'generation']]
    df['GHI'].plot(ax=ax, figsize=(15,8), color = 'tab:red', style={'GHI': '-o'}, legend=False)
    df[b'generation'].plot(ax=ax2, style={b'generation': '-o'})
    ax.set_ylabel('W/m2')
    ax2.set_ylabel('kW')
    ax.grid()
    ax.legend(loc=2, ncol=5, frameon=False)
    ax2.legend(loc=1, frameon=False)
    ax.set_xticks(df.index)
    if start_year_choice.value != end_year_choice.value:
        plt.title(f'Aggregated Monthly solar simulation results in {state_choice.value} from {start_year_choice.value} to {end_year_choice.value}')
    else:
        plt.title(f'Aggregated Monthly solar simulation results in {state_choice.value} in {start_year_choice.value}')

In [ ]:
def yearly_plot(df):
    # Using a style
    plt.style.use('Solarize_Light2')
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax2 = ax.twinx()
    df = df.groupby('Year').sum()[['GHI', b'generation']]
    df['GHI'].plot(ax=ax, figsize=(15,8), color = 'tab:red', style = {'GHI' : '-o'}, legend=False)
    df[b'generation'].plot(ax=ax2, style={b'generation': '-o'})
    ax.grid()
    ax.set_ylabel('W/m2')
    ax2.set_ylabel('kW')
    ax.legend(loc=2, ncol=5, frameon=False)
    ax2.legend(loc=1, frameon=False)
    ax.set_xticks(df.index)
    if start_year_choice.value != end_year_choice.value:
        plt.title(f'Aggregated Yearly solar simulation results in {state_choice.value} from {start_year_choice.value} to {end_year_choice.value}')
    else:
        plt.title(f'Aggregated Yearly solar simulation results in {state_choice.value} in {start_year_choice.value}')

In [ ]:
createParametersOptionsWidget()

In [ ]:
solar_power_simulation()

In [ ]:
daily_plot(final_data)

In [ ]:
monthly_plot(final_data)

In [ ]:
yearly_plot(final_data)